In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 28 20:52:23 2019

@author: Birdman
"""

# -*- coding: utf-8 -*-
"""
Created on Wed Sep 25 15:50:06 2019

@author: Birdman
"""

from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
import nltk
from sklearn import tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import precision_score, recall_score,accuracy_score
import csv
import pandas as pd

con_alpha = ['b','c','d','f','g','h','j','k','l','m','n','p','q','r','s','t','v','w','x','y','z']#辅音字母
Uni_alpha = ['a','e','i','o','u']#元音字母

/Users/greglamex/anaconda3/envs/scikit-learn/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def get_bigram(url):
    
    bigram_finder = BigramCollocationFinder.from_words(url)
    bigram_BAM = nltk.collocations.BigramAssocMeasures()
    bi_scored = bigram_finder.score_ngrams(bigram_BAM.pmi)
    #print(scored)
    result = 0
    for i in bi_scored:
        result = result + i[1]
    return int(result)
    

In [3]:
def get_trigram(url):
    
    trigram_finder = TrigramCollocationFinder.from_words(url)
    trigram_TAM = nltk.collocations.TrigramAssocMeasures()
    scored = trigram_finder.score_ngrams(trigram_TAM.poisson_stirling)
    result = 0
    for i in scored:
        result = result + i[1]
    return int(result)

In [4]:
def if_or_not_in_con_alpha(index):
    if index in con_alpha:
        return 1
    else:
        return 0
    

In [5]:
#计算元音字母出现频率
def count_uni_alpha(url):
    i = 0
    for index in Uni_alpha:
        i = i + url.count(index)
    return(i/(len(url)))
    

In [6]:
#计算元音字母出现频率
def count_con_alpha(url):
    total = 0
    for index in range(0,len(url)):
        while if_or_not_in_con_alpha(url[index]) == 1 and index <len(url)-1:
            index = index + 1
            if if_or_not_in_con_alpha(url[index]) == 0 or index == len(url)-1:
                total = total + 1
                break
    return(total)

In [7]:
#统计url中数字出现的次数
def count_number(url):
    total = 0
    for i in url:
        if ord(i)>=48 and ord(i)<=57:
            total = total + 1
    return(total)

In [8]:
#信息熵
def calc_ent(url):
    data = []
    for i in url:
        data.append(i)
    x = np.array(data)
    x_value_list = set([x[i] for i in range(x.shape[0])])#得到数组x的元素（不包含重复元素），即x_value_list={'c', 'b', 'a'}
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]#计算每个元素出现的概率
        logp = np.log2(p)
        ent -= p * logp
    return(ent)

In [9]:
def url_length(url):
    return(len(url))

In [10]:
def get_features(temp,url):
    temp.append(calc_ent(url))
    temp.append(count_uni_alpha(url))
    temp.append(count_con_alpha((url)))
    temp.append(count_number(url))
    temp.append(url_length(url))
    temp.append(get_bigram(url))
    temp.append(get_trigram(url))
    return temp

In [11]:
url = []
label = []
temp = []
data = []
LaBel = []

with open(r'top-1m.csv', 'r', encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        url.append(row['url'])
        label.append(0)  # 健康域名
    csvfile.close()
with open(r'dga.csv', 'r', encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        url.append(row['url'])
        label.append(1)  # dga域名
    csvfile.close()

# print(url[:10])
# print(label[:10])

for i in range(0, len(url)):
    # for i in range(0,10):
    temp = get_features(temp, url[i])
    data.append(temp)
    LaBel.append(label[i])
    temp = []

data = np.array(data)
LaBel = np.array(LaBel)
# 保存特征数据

print(data[:25])
print(LaBel[:25])


[[  3.45943162   0.27272727   6.           0.          11.
   34.          53.        ]
 [  3.879664     0.31818182  11.           0.          22.
   69.         123.        ]
 [  3.75         0.25         9.           0.          16.
   52.          87.        ]
 [  3.97541802   0.23809524  12.           0.          21.
   71.         124.        ]
 [  2.64643934   0.4          4.           0.          10.
   17.          30.        ]
 [  3.02698683   0.30769231   7.           0.          13.
   28.          49.        ]
 [  3.61634857   0.36842105   9.           0.          19.
   53.          94.        ]
 [  2.84237099   0.28571429   7.           0.          14.
   24.          45.        ]
 [  3.37355726   0.2          9.           0.          15.
   39.          67.        ]
 [  3.68418372   0.2         12.           0.          20.
   56.         100.        ]
 [  3.68385619   0.16        16.           0.          25.
   64.         122.        ]
 [  3.73452166   0.35294118   9.

In [12]:
# test
X_train, X_test, y_train, y_test = train_test_split(data, LaBel, test_size=0.3, random_state=0)
sc = StandardScaler()
sc.fit(X_train)
# 对数据集进行处理
dga_tree = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2,
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=7,
                                       random_state=None, max_leaf_nodes=None,
                                       min_impurity_decrease=0.0, min_impurity_split=None, presort=False)

print("***************start****************")

dga_tree_save = dga_tree.fit(X_train, y_train)
score = cross_val_score(dga_tree, data, LaBel, cv=5)
print(score)





***************start****************
[0.89989996 0.95031416 0.6137503  0.63379148 0.68794202]


In [13]:
print("**************saveing***************")
# joblib.dump(dga_tree_save,'C:/Users/Birdman/Desktop/data/dga/dga.model')
print("Training score:%f" % (dga_tree.score(X_train, y_train)))
print("Testing score:%f" % (dga_tree.score(X_test, y_test)))
y_pred = dga_tree.predict(X_test)
print("Testing precision score:%f" % (accuracy_score(y_test, y_pred)))
print("Testing precision score:%f" % (precision_score(y_test, y_pred)))
print("Testing recall score:%f" % (recall_score(y_test, y_pred)))


**************saveing***************
Training score:0.959589
Testing score:0.948114
Testing precision score:0.948114
Testing precision score:0.953263
Testing recall score:0.952393
